# Head

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/MambaVAE")

Mounted at /content/gdrive


In [ ]:
os.environ['MAX_JOBS'] = '3'
%pip install ninja
%pip install --no-build-isolation --verbose ./selective_scan

In [ ]:
import torch, InitMamba
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

# Lab

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
inputs = tokenizer(["Today is Monday, Tomorrow is Tuesday.",
           "Today is Sunday, Tomorrow is Monday.",
           "Today is Sunday, Tomorrow is Monday."],
          #   max_length = 32,
          #  padding = 'max_length',
           return_tensors='pt').to('cuda')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

In [ ]:
model1 = InitMamba.MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf").cuda()
model1.train()

res1 = model1(**inputs, output_ssm_last_states = True)
ssm_last_states = res1.ssm_last_states
print(ssm_last_states[-1])

config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/517M [00:00<?, ?B/s]

The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. For the mamba.py backend, follow https://github.com/alxndrTL/mamba.py.


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tensor([[[ 2.1797e-02, -9.7853e-02,  5.0647e-03,  ...,  7.4099e-02,
          -1.4307e-01, -1.8233e-01],
         [ 7.4937e-03, -1.0742e-02,  5.7249e-04,  ..., -8.5048e-03,
           1.3370e-02,  2.0914e-02],
         [-7.4588e-02,  8.4493e-02, -2.8190e-03,  ..., -3.7165e-02,
           8.2320e-02,  1.0191e-01],
         ...,
         [ 9.4912e-03, -1.3890e-02,  2.5660e-03,  ...,  2.9801e-02,
          -6.6740e-02, -8.9122e-02],
         [-1.1792e-01,  5.8620e-02, -9.2658e-03,  ..., -1.1675e-01,
           2.1422e-01,  2.8709e-01],
         [-4.2002e-03,  6.3560e-03, -9.7593e-04,  ..., -6.9139e-03,
           2.5129e-02,  3.5111e-02]],

        [[ 2.4119e-02, -9.5605e-02,  6.1495e-03,  ...,  7.5622e-02,
          -1.4609e-01, -1.8936e-01],
         [ 8.2959e-03, -1.2948e-02,  1.5957e-04,  ..., -3.7550e-03,
           6.0886e-03,  9.3989e-03],
         [-7.3239e-02,  8.2168e-02, -3.4553e-03,  ..., -3.9539e-02,
           8.6269e-02,  1.0851e-01],
         ...,
         [ 9.7279e-03, -1

In [ ]:
res = model1(**inputs, output_ssm_last_states = True, inputs_ssm_states = ssm_last_states)
print(res.logits)
tokenizer.batch_decode(res.logits.argmax(-1))

tensor([[[-50.6910, -66.3722, -47.7313,  ..., -66.4962, -66.2791, -66.5470],
         [-51.3856, -67.4401, -50.2360,  ..., -67.6237, -67.5241, -67.5715],
         [-30.5410, -48.7618, -29.7472,  ..., -48.6926, -48.7306, -48.9155],
         ...,
         [ -5.1940, -24.0815,  -6.4142,  ..., -24.1863, -24.3237, -24.1754],
         [-34.1231, -55.2161, -34.2415,  ..., -55.1909, -55.1099, -55.3477],
         [-40.5455, -60.5548, -46.9419,  ..., -60.6998, -60.5753, -60.7300]],

        [[-50.5576, -66.2595, -47.6861,  ..., -66.3801, -66.1592, -66.4360],
         [-41.8694, -58.5675, -40.6945,  ..., -58.7701, -58.6720, -58.6985],
         [-39.3041, -57.5434, -38.2461,  ..., -57.5422, -57.4609, -57.6745],
         ...,
         [-13.2787, -32.1403, -14.9096,  ..., -32.1918, -32.2581, -32.2140],
         [-33.7567, -54.3394, -34.8080,  ..., -54.2912, -54.1922, -54.4791],
         [-43.9088, -64.0019, -50.8268,  ..., -64.1493, -64.0093, -64.1931]],

        [[-50.5576, -66.2595, -47.6861,  ...

[' is Wednesday, Tomorrow is Tuesday,\n',
 ' is Tuesday, Tomorrow is Monday,\n',
 ' is Tuesday, Tomorrow is Monday,\n']

In [ ]:
torch.autograd.grad(outputs=res.logits.mean(), inputs=ssm_last_states[0])

(tensor([[[ 0.0000e+00, -1.6750e-04, -1.7511e-02,  ..., -3.1498e-15,
           -3.1184e-08,  2.0424e-16],
          [ 4.1588e-02, -1.2297e-06, -2.5141e-04,  ..., -2.5786e-05,
           -0.0000e+00,  2.1951e-02],
          [-3.2010e-12,  4.2407e-03,  4.1456e-02,  ...,  1.9357e-24,
            1.5062e-12, -0.0000e+00],
          ...,
          [-4.1978e-05,  1.2930e-18,  2.8190e-03,  ..., -3.3347e-02,
           -1.2521e-08,  1.5717e-10],
          [ 4.1835e-02, -4.3419e-02,  4.0228e-33,  ..., -5.9103e-05,
            0.0000e+00, -0.0000e+00],
          [ 1.0688e-02, -9.9554e-05, -7.5545e-03,  ..., -1.2935e-04,
           -0.0000e+00,  5.7639e-03]],
 
         [[ 0.0000e+00, -1.6721e-04,  7.5137e-03,  ..., -3.2217e-15,
           -3.1859e-08,  2.0891e-16],
          [ 2.0194e-02, -5.4066e-07, -1.0819e-04,  ..., -1.1341e-05,
           -0.0000e+00,  1.2631e-02],
          [-3.2291e-12,  3.9676e-03,  3.5296e-02,  ...,  1.9527e-24,
            1.5194e-12, -0.0000e+00],
          ...,
    

In [ ]:
from google.colab import runtime
runtime.unassign()